# Working with Data APIs

This lecture material was originally developed by Sam Maurer in 2017, and has been updated.

This notebook provides a demonstration of data-access APIs that operate over the web. See README.md for setup instructions.

In Part 1, we'll load and parse results from an API feed of earthquake data.  
In Part 2, we'll add query parameters to the workflow, using the Mapbox Geolocation API as an example.  
In Part 3, we'll use an authenticated API to query public Twitter posts. 

Before jumping in, let's quickly scan the range of APIs that are available:

* Socrata Open Data: https://dev.socrata.com/
  * San Francisco: https://datasf.org/opendata/
  * Berkeley: https://data.cityofberkeley.info/

* Transitland GTFS Feeds: https://transit.land/
* Ford Bike Share: https://www.fordgobike.com/system-data

* US Census Bureau: https://www.census.gov/data/developers/data-sets.html
* Bureau of Labor Statistics: https://www.bls.gov/developers/
* US Geological Survey: https://www.usgs.gov/products/data-and-tools/apis
* US Environmental Protection Agency: https://www.epa.gov/enviro/web-services

* Google APIs: https://console.developers.google.com/apis/library?project=cp255-185021
* Facebook: https://developers.facebook.com/docs/apis-and-sdks/
* Twitter: https://developer.twitter.com/en/docs/api-reference-index.html
* Foursquare: https://developer.foursquare.com/
* Instagram: https://www.instagram.com/developer/
* Yelp: https://www.yelp.com/developers

# Part 1: Reading from an automated data feed

### USGS real-time earthquake feeds

This is an API for near-real-time data about earthquakes. Data is provided in JSON format over the web. No authentication is needed, and there's no way to customize the output. Instead, the API has a separate endpoint for each permutation of the data that users might want.

**API documentation:**  
http://earthquake.usgs.gov/earthquakes/feed/v1.0/geojson.php

**Sample API endpoint, for magnitude 4.5+ earthquakes in past day:**  
http://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/4.5_day.geojson  


In [1]:
%matplotlib inline

import pandas as pd

import json      # library for working with JSON-formatted text strings
import requests  # library for accessing content from web URLs

import pprint    # library for cleanly printing Python data structures
pp = pprint.PrettyPrinter()

In [2]:
# download data on magnitude 2.5+ quakes from the past week

endpoint_url = "http://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_week.geojson"
response = requests.get(endpoint_url)
results = response.text

# what is the data type of the results?

print(type(results))

<class 'str'>


In [3]:
# print the first 500 characters to see a sample of the data

print(results[:500])

{"type":"FeatureCollection","metadata":{"generated":1539195051000,"url":"https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_week.geojson","title":"USGS Magnitude 2.5+ Earthquakes, Past Week","status":200,"api":"1.5.8","count":270},"features":[{"type":"Feature","properties":{"mag":2.88,"place":"3km SW of Anderson Springs, CA","time":1539190685110,"updated":1539192723816,"tz":-480,"url":"https://earthquake.usgs.gov/earthquakes/eventpage/nc73095881","detail":"https://earthquake.usgs.gov/


In [4]:
# it looks like the results are a string with JSON-formatted data inside

# parse the string into a Python dictionary (loads = "load string")
data = json.loads(results)

print(type(data))

<class 'dict'>


In [ ]:
# print the dictionary

pp.pprint(data)

In [ ]:
# subsets of the data can be accessed with square brackets, using labels for the 
# named elements and numerals for the lists

# save the list of quakes to a new variable

quakes = data['features']

# print the most recent quake

pp.pprint(quakes[0])

In [ ]:
# pull out the title from each earthquake listing

for q in quakes:
    print(q['properties']['title'])

In [ ]:
# pull out magnitudes and depths into a Pandas dataframe, using
# a more compact Python syntax for iterating through lists

d = {'magnitude': [q['properties']['mag'] for q in quakes],
     'depth': [q['geometry']['coordinates'][2] for q in quakes]}

df = pd.DataFrame.from_dict(d)

# how many earthquakes were loaded into the dataframe?

print(len(df))

In [ ]:
# print the first few lines of data

print(df.head())

In [ ]:
# print some descriptive statistics

print(df.describe())

In [ ]:
# plot the depth vs. magnitude

df.plot(x='magnitude', y='depth', kind='scatter')

In [ ]:
# save the dataframe to disk

df.to_csv('usgs_earthquake_data.csv')

print('file saved')

In [ ]:
# read it back later

new_df = pd.read_csv('usgs_earthquake_data.csv')

print(new_df.head())

# Part 2: Querying an API endpoint

### Mapbox Geocoding API

Services like Google Maps and Mapbox have various APIs that let you access its services through code instead of through GUI apps. This one from Mapbox lets you look up the latitude-longitude coordinates of street addresses.

It works similarly to the earthquakes example, but with query parameters added to the URL endpoint!

**API documentation:**  
https://www.mapbox.com/api-documentation/#geocoding

**API endpoint:**  
https://api.mapbox.com/geocoding/v5/mapbox.places

**API endpoint with query parameters:**  
https://api.mapbox.com/geocoding/v5/mapbox.places/Wurster+Hall.json?access_token=pk.eyJ1IjoiY3AyNTVkZW1vIiwiYSI6ImRPcTlnTUEifQ.3C0d0Nk_rcwV-8JF29PU-w

You can get your own access key by signing up for a Mapbox account, if you'd like. Here is a link for that (but don't do it now): https://www.mapbox.com/signin/?route-to=%22/account/access-tokens%22

In [ ]:
import json      # library for working with JSON-formatted text strings
import requests  # library for accessing content from web URLs

import pprint    # library for cleanly printing Python data structures
pp = pprint.PrettyPrinter()

In [ ]:
# we have to encode the search query so that it can be passed as a URL, 
# with spaces and other special characters removed

endpoint = 'https://api.mapbox.com/geocoding/v5/mapbox.places/'

address = 'Wurster Hall'

params = {'limit': 1,
          'access_token': 'pk.eyJ1IjoiY3AyNTVkZW1vIiwiYSI6ImRPcTlnTUEifQ.3C0d0Nk_rcwV-8JF29PU-w'}

url = requests.Request('GET', endpoint+address+'.json', params=params).prepare().url
print(url)

In [ ]:
# download and parse the results

response = requests.get(url)
results = response.text
data = json.loads(results)

print(data)

In [ ]:
# print it more nicely

pp.pprint(data)

In [ ]:
# pull out the lat-lon coordinates

for r in data['features']:
    coords = r['geometry']['coordinates']
    print(coords)

### Exercises

1. Search for some other addresses or landmarks!
2. Take a look at the [API documentation](https://www.mapbox.com/api-documentation/#geocoding). Can you figure out how to retrieve other points of interest near Wurster Hall?

# Part 3: Querying an API with back-and-forth authentication

### Twitter search APIs

Twitter's APIs operate over the web as well, but they require a back-and-forth authentication process at the beginning of each connection. It's easier to have a Python library handle this than to create the query URLs ourselves.

The Twitter "REST" APIs perform stand-alone operations: you submit a query and receive results, like in earlier examples. ([REST](https://en.wikipedia.org/wiki/Representational_state_transfer) is a particular set of guidelines that many APIs follow.) Twitter also has a "streaming" API that continues sending results in real time until you disconnect.

**API documentation:**  
https://dev.twitter.com/rest/public  
https://dev.twitter.com/overview/api/tweets

**Documentation for the Python helper library**:  
http://geduldig.github.io/TwitterAPI/

In [ ]:
#!pip install TwitterAPI

In [ ]:
from TwitterAPI import TwitterAPI

import pprint
pp = pprint.PrettyPrinter()

In [ ]:
# import API credentials from keys.py file in the
# same directory as this notebook

from keys import *

In [ ]:
# set up an API connection using credentials from the keys file

api = TwitterAPI(consumer_key, consumer_secret, 
                 access_token, access_token_secret)

print("Connection is set up but not tested")

### Making a simple data request

In [ ]:
# most recent tweet from CED

endpoint = 'statuses/user_timeline'
params = {
    'screen_name': 'wursterlife', 
    'count': 1
}
r = api.request(endpoint, params)

for tweet in r.get_iterator():
    print(tweet['text'])

In [ ]:
# what other data is there?

pp.pprint(tweet)

### Other API endpoints allow different types of searches

In [ ]:
# search for recent tweets with your favorite emoji

endpoint = 'search/tweets'
params = {
    'q': '👻🎃', 
    'count': 5
}
r = api.request(endpoint, params)

for tweet in r.get_iterator():
    print(tweet['text'] + '\n')

In [ ]:
# search for public tweets in Korean

endpoint = 'search/tweets'
params = {
    'q': '*', 
    'lang': 'ko', 
    'count': 5
} 
r = api.request(endpoint, params)

for tweet in r.get_iterator():
    print(tweet['text'] + '\n')

In [ ]:
# search for public tweets geotagged near the UC Berkeley campus

endpoint = 'search/tweets'
params = {
    'q': '*', 
    'geocode': '37.873,-122.260,0.5km', 
    'count': 5
} 
r = api.request(endpoint, params)

for tweet in r.get_iterator():
    print(tweet['text'] + '\n')

### Exercises

1. Try some different search queries!
2. Display some more data fields in addition to the tweet text

### Bonus: Streaming live tweets in real time 

Check in: http://bitly.com/cp255

In [ ]:
# Twitter allows only one or two simultaneous streaming connections for 
# each set of API credentials, so this part may not work during class

endpoint = 'statuses/filter'
params = {'locations': '-180,-90,180,90'}
r = api.request(endpoint, params)
LIMIT = 20

# 'enumerate' lets us count tweets as we receive them

for i, tweet in enumerate(r.get_iterator()):
    print(tweet['created_at'])
    print(tweet['place']['full_name'] + ', ' + tweet['place']['country'])
    print(tweet['text'] + '\n')
    if (i > LIMIT): break

# close the streaming connection
r.close()

### Exercises for the remainder of class

Choose your favorite:

1. Make a scatter plot of the lat-lon coordinates of earthquakes.  
   &nbsp;
   
2. The earthquakes API is actually returning a specific data format called [GeoJSON](https://en.wikipedia.org/wiki/GeoJSON), also used by many other geospatial data feeds. Try saving the raw GeoJSON file and opening it in QGIS or in the [geojson.io](http://geojson.io) web viewer.  
   &nbsp;

2. Using the geocoding example as a starting point, try searching Mapbox's Directions API or Isochrone (travel time contours) API instead. You can read more about them on the [Mapbox API documentation page](https://www.mapbox.com/api-documentation/#introduction).  
    &nbsp;

3. Try out another API that you're interested in. Can you figure out how to connect to it using Python?  

   With municipal data it's often easiest to just download a data file, but APIs are great for querying big data sets or tracking live updates. Here are some resources.

   - San Francisco:  https://data.sfgov.org/developers  
   - Alameda County:  https://data.acgov.org/developers  
   - UC Berkeley:  https://api-central.berkeley.edu  
   - US Census:  http://www.census.gov/data/developers/data-sets.html  
   - Open Data Network:  https://www.opendatanetwork.com  
   - CivicData:  http://www.civicdata.io/  